# Custom Class Example

This example uses several other python files in the same folder.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

In [2]:
from custom_app import CustomApp # our custom app

from trulens_eval.tru_custom_app import TruCustomApp

from trulens_eval import Tru
# Tru object manages the database of apps, records, and feedbacks; and the
# dashboard to display these.
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
Deleted 2 rows.


In [3]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # Optional alternative to decorators:
    #methods_to_instrument={
    #    ca.respond_to_query: Select.Query(), # paths relative to "app"
    #    ca.retrieve_chunks: Select.Query(),
    #    ca.retriever.retrieve_chunks: Select.Query().retriever
    #},
    ## Add extra data to show up as app serialization. See tru_custom_app.py documentation.
    #app_extra_json=dict(
    #    name="This is my custom app. Anything provided to app_extra_json will be merged into the serialization of app",
    #)
)


In [4]:
# Show instrumented components and methods.

ta.print_instrumented()

Components:
Custom of trulens_eval.app component: *.__app__.app
Custom of trulens_eval.app component: *.__app__.app.llm
Custom of trulens_eval.app component: *.__app__.app.retriever
Custom of trulens_eval.app component: *.__app__.app.template

Methods:
Object at 0x176f76280:
	<function CustomApp.retrieve_chunks at 0x176f18550> with path *.__app__.app
	<function CustomApp.respond_to_query at 0x176f185e0> with path *.__app__.app
Object at 0x176fc49d0:
	<function CustomLLM.generate at 0x176efcaf0> with path *.__app__.app.llm
Object at 0x176f765b0:
	<function CustomRetriever.retrieve_chunks at 0x176f18160> with path *.__app__.app.retriever
Object at 0x176fc47f0:
	<function CustomTemplate.fill at 0x176f18310> with path *.__app__.app.template


In [5]:
# Normal usage:
# ca.respond_to_query("What is the capital of Indonesia?")

# Instrumented usage:
response, record = ta.with_record(
    ca.respond_to_query, input="What is the capital of Indonesia?"
)


creating new recording context
hello <function CustomApp.respond_to_query at 0x176f185e0>
hello <function CustomApp.retrieve_chunks at 0x176f18550>
hello <function CustomRetriever.retrieve_chunks at 0x176f18160>


Unsure what the main output string is for the call to retrieve_chunks.


_on_add_record, bindings=<BoundArguments (self=<custom_retriever.CustomRetriever object at 0x176f765b0>, data='What is the capital of Indonesia?')>
adding a record
_on_add_record, bindings=<BoundArguments (self=<custom_app.CustomApp object at 0x176f76280>, data='What is the capital of Indonesia?')>
hello <function CustomLLM.generate at 0x176efcaf0>
_on_add_record, bindings=<BoundArguments (self=<custom_llm.CustomLLM object at 0x176fc49d0>, prompt='Relevant chunk: WHAT IS THE CAPITAL OF INDONESIA?,Relevant chunk: ?aisenodnI fo latipac eht si tahW')>
hello <function CustomTemplate.fill at 0x176f18310>
_on_add_record, bindings=<BoundArguments (self=<custom_app.CustomTemplate object at 0x176fc47f0>, question='What is the capital of Indonesia?', answer='herp What is the capital of Indonesia? :knuhc tnaveleR,?AISENODNI FO LATIPAC EHT SI TAHW :knuhc tnaveleR derp')>
_on_add_record, bindings=<BoundArguments (self=<custom_app.CustomApp object at 0x176f76280>, input='What is the capital of Indon

In [6]:
# Show the app output:

response

'The answer to What is the capital of Indonesia? is probably herp What is the capital of Indonesia? :knuhc tnaveleR,?AISENODNI FO LATIPAC EHT SI TAHW :knuhc tnaveleR derp or something ...'

In [7]:
# Show the instrumentation record.

record.dict()

{'record_id': 'record_hash_527bcbcd8c15d46946feeb93e35d9222',
 'app_id': 'customapp',
 'cost': {'n_requests': 0,
  'n_successful_requests': 0,
  'n_classes': 0,
  'n_tokens': 0,
  'n_stream_chunks': 0,
  'n_prompt_tokens': 0,
  'n_completion_tokens': 0,
  'cost': 0.0},
 'perf': {'start_time': datetime.datetime(2023, 8, 22, 16, 1, 22, 446457),
  'end_time': datetime.datetime(2023, 8, 22, 16, 1, 22, 956331)},
 'ts': datetime.datetime(2023, 8, 22, 16, 1, 22, 957106),
 'tags': '-',
 'main_input': 'What is the capital of Indonesia?',
 'main_output': 'Relevant chunk: WHAT IS THE CAPITAL OF INDONESIA?',
 'main_error': None,
 'calls': [{'stack': ({'path': {'path': ({'item_or_attribute': 'app'},
       {'item_or_attribute': 'retriever'})},
     'method': {'obj': {'cls': {'name': 'CustomRetriever',
        'module': {'package_name': '', 'module_name': 'custom_retriever'},
        'bases': None},
       'id': 6290892208},
      'name': 'retrieve_chunks'}},),
   'args': {'data': 'What is the capit

In [8]:
# Start the dasshboard. If you running from github repo, you will need to adjust
# the path the dashboard streamlit app starts in by providing the _dev argument.
tru.start_dashboard(
    force=True, _dev=Path().cwd().parent.parent.parent.resolve()
)

Force stopping dashboard ...
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.86.40:8501 .
